In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv")
train.head()
train.shape

In [ ]:
#Parameters
oov_token = "<OOV>"
num_words = 10000
embedding_dim = 32
max_length = 50
train_val_split = 0.75

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
tweets_raw = train['text'].tolist()
labels = train['sentiment'].tolist()

tweets = []

for t in tweets_raw:
    tweets.append(str(t))
    
tweets_train = tweets[:np.int(len(tweets) * train_val_split)]
tweets_valid = tweets[np.int(len(tweets) * train_val_split):]

labels_train = labels[:np.int(len(tweets) * train_val_split)]
labels_valid = labels[np.int(len(tweets) * train_val_split):]

In [ ]:
tokenizer =  Tokenizer(num_words = num_words, oov_token=oov_token)
tokenizer.fit_on_texts(tweets_train)
word_index = tokenizer.word_index
print(len(word_index))

In [ ]:
sequences_train = tokenizer.texts_to_sequences(tweets_train)
padded_train = pad_sequences(sequences_train, maxlen=max_length)

sequences_valid = tokenizer.texts_to_sequences(tweets_valid)
padded_valid = pad_sequences(sequences_valid, maxlen=max_length)

In [ ]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels_train)

label_word_index = label_tokenizer.word_index

train_label = np.array(label_tokenizer.texts_to_sequences(labels_train))
train_label = train_label - 1
train_label = to_categorical(train_label, num_classes = 3)

valid_label = np.array(label_tokenizer.texts_to_sequences(labels_valid))
valid_label = valid_label - 1
valid_label = to_categorical(valid_label, num_classes = 3)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = num_words, output_dim = embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])
model.summary()

In [ ]:
history = model.fit(padded_train, train_label, epochs = 10, validation_data = (padded_valid, valid_label))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#print(history.history)

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

epoch = range(len(loss))

plt.plot(epoch, loss)
plt.plot(epoch, val_loss)
plt.figure()

plt.plot(epoch, acc)
plt.plot(epoch, val_acc)
plt.figure()